# Imports

In [ ]:
import os
import pandas as pd
import datetime

## load sample raw data


In [ ]:
import pickle
path_to_data = os.path.join("..", "data")
sample_fname = "sample_raw.pkl"

with open(os.path.join(path_to_data, sample_fname), "rb") as file:
    raw_data = pickle.load(file)

raw_data[:1]

## create cleaning pipeline

steps:
- convert games from list/dict to pandas Series
- convert dates to pd.datetimes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
USERNAME = "JammyNinja"
def game_to_pandas(game):
    """ 
        converts a game as received from chess.com API JSON to pandas Series

        try out the raw form at: 
            https://api.chess.com/pub/player/jammyninja/games/2023/01
    """
    display(game)
    game_dict = {}
    
    game_pgn = game["pgn"]
    game_dict["result"] = game_pgn.split("}")[-1].strip()
    game_dict['time_control'] = game["time_control"]
    game_dict['time_class'] = game["time_class"]
    game_dict['rules'] = game["rules"] #to exclude any chess960 or other that may have been played

    game_dict['start_time'] = game["start_time"]

    user_colour = "white" if game["white"]["username"] == USERNAME else "black"
    opp_colour = "black" if user_colour == "white" else "white" #used to save time below

    game_dict["user_colour"] = user_colour
    game_dict['opp_username'] = game[opp_colour]["username"]
    
    game_dict['user_rating'] = game[user_colour]["rating"]
    game_dict['opp_rating'] = game[opp_colour]["rating"]

    game_dict['user_result'] = game[user_colour]["result"]
    game_dict['opp_result'] = game[opp_colour]["result"]
    
    return game_dict
    


test_game_dict = game_to_pandas(raw_data[0])
test_game_dict

In [ ]:
def get_date(game_dict):
    print(datetime.datetime.fromtimestamp(game_dict['start_time']).strftime('%d-%m-%Y'))
    return
                                                                      
get_date(test_game_dict)

In [ ]:
def date_conversion(df):
    dt_cols = ["date", "start_time", "end_time"]
    for col in dt_cols:
        df[col] = pd.to_datetime(df[col])
    return df

date_conversion(raw_data).dtypes

In [ ]:
date_converter = FunctionTransformer(date_conversion)

clean_pipe = Pipeline([
    #("json to dict")
    #(dict to pandas)
    ("date dtype", date_converter)
])

In [ ]:
clean_pipe.fit_transform(raw_data).dtypes

In [ ]:
test_data = [0,1,2,3,4,5]

test_pipe = Pipeline([
    ("doubler", FunctionTransformer(lambda x : x*2))
])

In [ ]:
test_pipe.fit_transform(test_data)

### create preproc pipeline

add new columns